In [ ]:
import numpy as np # linear algebra
import pandas as pd
# from tqdm import tqdm
from tqdm import tqdm_notebook as tqdm
# tqdm.pandas()

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Flatten, TimeDistributed, LeakyReLU, Conv1D, BatchNormalization, MaxPooling1D, AveragePooling1D
from tensorflow.keras.models import Model, Sequential
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.keras import layers, optimizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2

from xgboost import XGBClassifier, plot_importance

import pickle
import Model_function as func

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
cluster = 0
with open(f'D:\\庫存健診開發\\data\\Training\\processed\\Cluster_{cluster}_classification_minmax0_Weekly', 'rb') as fp:
    load_list = pickle.load(fp)

X_train, Y_train, X_test, Y_test = load_list[0], load_list[1], load_list[2], load_list[3]
X_train, Y_train, X_val, Y_val = func.validation_split(X_train, Y_train)
y_train, y_val, y_test = func.get_dependent_variable(Y_train, Y_val, Y_test)
y_train_encode, y_val_encode, y_test_encode = func.categorical_transform(y_train), func.categorical_transform(y_val), func.categorical_transform(y_test)

In [ ]:
class CNN_Tree_Classifier():
    def __init__(self, train, val, test, classifier):
        self.X_train = np.array(train[0])
        self.Y_train = np.array(train[1])
        self.X_val = np.array(val[0])
        self.Y_val = np.array(val[1])
        self.X_test = np.array(test[0])
        self.Y_test = np.array(test[1])
        self.classifier = classifier.lower()
        self.param1 = {}
        self.param2 = {}

        
    def CNN_param(self, param):
        self.param1 = param
    
    def CNN(self):
        model = Sequential()
        model.add(Conv1D(
                filters=self.param1['l1_filter'], 
                kernel_size=self.param1['l1_kernel'], 
                activation='relu', 
                input_shape=(self.X_train.shape[1], self.X_train.shape[2]), 
                padding=self.param1['padding1']))
        
        model.add(Conv1D(
                filters=self.param1['l2_filter'], 
                kernel_size=self.param1['l2_kernel'], 
                activation='relu',
                padding=self.param1['padding2']))
            
        if (self.param1['dropout']) and (self.param1['Conv_layer'] <= 2):
            model.add(Dropout(self.param1['dropout_ratio']))
        
        if self.param1['Conv_layer'] > 2:
            model.add(Conv1D(
                filters=self.param1['l3_filter'], 
                kernel_size=self.param1['l3_kernel'], 
                activation='relu',
                padding=self.param1['padding3']))
            if self.param1['dropout']:
                model.add(Dropout(self.param1['dropout_ratio']))
                
        model.add(Flatten(name='feature'))
        
        if self.param1['Dense']:
            model.add(Dense(self.param1['Dense']))
            model.add(Dropout(self.param1['dropout_ratio']))
            
        model.add(Dense(2, activation='softmax'))
        model.compile(loss='categorical_crossentropy', optimizer=optimizers.Nadam(lr=0.001), metrics=[tf.keras.metrics.AUC()])
        self.CNN_ = model.fit(self.X_train, self.Y_train, validation_data=(self.X_val, self.Y_val), batch_size=self.param1['batchsize'], epochs=self.param1['epochs'], verbose=2)
        
    def Feature_extraction(self):
        intermediate_layer_model = Model(inputs=self.CNN_.input, outputs=self.CNN_.get_layer('feature').output)
        self.training_feature_ = intermediate_layer_model.predict(self.X_train)
        self.val_feature_ = intermediate_layer_model.predict(self.X_val)
        self.testing_feature_ = intermediate_layer_model.predict(self.X_test)
        
    def Classifier_param(self, param):
        self.param2 = param
        
    def Classifier(self):
        if self.classifier == 'randomforest':
            classifier = RandomForestClassifier(n_estimators = 400, max_depth=20, verbose=1, n_jobs=-1)
            self.model_ = forest.fit(self.training_feature_, self.y_train)
        elif self.classifier == 'xgboost':
            classifier = XGBClassifier()
            self.model_ = classifier.fit(self.training_feature_, self.y_train, eval_set = [(self.training_feature_, self.y_train), (self.val_feature_, self.y_train)], verbose=True, eval_metric='auc')

        else:
            return 'Classifier not exist'
        
    
    
        